In [11]:
from sleap_roots.series import find_all_series, Series
from sleap_roots.trait_pipelines import MultipleDicotPipeline, Pipeline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import os

from typing import List
from pathlib import Path

In [12]:
# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print("Current directory:", current_directory)

Current directory: d:\elizabeths_repos\sleap-roots\notebooks


In [13]:
# Define the directory you want to change to
new_directory = "D:/elizabeths_repos/sleap-roots"

# Change the current working directory
os.chdir(new_directory)

# Get the updated current working directory
updated_directory = os.getcwd()

# Print the updated current working directory
print("Updated current directory:", updated_directory)


Updated current directory: D:\elizabeths_repos\sleap-roots


In [14]:
csv_path = "tests/data/multiple_arabidopsis_11do/merged_proofread_samples_03122024.csv" # For sample information (count, group)
folder_path = "tests/data/multiple_arabidopsis_11do" # Location of h5 files and predictions
primary_name = "primary" # For loading primary root predictions
lateral_name = "lateral" # For loading lateral root predictions

In [15]:
# Find all h5 files in the folder
all_h5s = find_all_series([folder_path])
all_h5s

['tests/data/multiple_arabidopsis_11do/6039_1.h5',
 'tests/data/multiple_arabidopsis_11do/7327_2.h5',
 'tests/data/multiple_arabidopsis_11do/9535_1.h5',
 'tests/data/multiple_arabidopsis_11do/997_1.h5']